# JAR625M - Week 9 - Practical 1

*Simon Matthews (simonm@hi.is)*

Import the required packages, including the main calculation tool that we will use: `pyDEW`

The `pyDEW` software implements the Deep Earth Water (DEW) model (e.g., Sverjensky, 2019) in python. The DEW model calculates the composition of fluids under crustal and mantle conditions.

In [ ]:
import pyDEW
from pyDEW import defaultsystem
import numpy as np
import pandas as pd

Define some useful functions for doing the calculations and giving results in a format that you can easily use for the calculations. You need to run this cell, but you can skip past what this code does if you want.

In [ ]:
atm_wt = np.array([0.0, 
                   1.008, 0.0, 0.0, 0.0, 0.0, 
                   0.0, 0.0, 15.999, 0.0, 0.0, 
                   22.99, 24.305, 26.982, 28.085, 0.0,
                   0.0, 35.45, 0.0, 39.098, 40.078,
                   0.0, 0.0, 0.0, 0.0, 0.0, 55.845,])

def run_reaction(fluid, T, P, bulk_composition, zimax):
    reaction = pyDEW.Reaction(fluid, T, P, 
                              bulk_composition=bulk_composition,
                              bulk_composition_units='wtpt_oxides',
                              mols_bulk_composition=zimax,
                              zimax=zimax)
    return reaction

def run_model(T, P, bulk_composition):
    # Get the default chemical system for pyDEW
    # This defines the minerals, and fluid properties
    # for the calculation.
    system = pyDEW.System()

    # Before running an equilibrium calculation we must
    # create a fluid that contains all the elements we
    # are interested in, in dilute quantities:
    molalities = {
        'H4SIO4(AQ)':0.01,
        'AL+3':0.01,
        'FE+2':0.01,
        'MG+2': 0.01,
        'CA+2': 0.01,
        'NA+': 0.01,
        'K+': 0.01,
        'CL-':0.01,
        }
    
    # Create this dilute fluid:
    fluid = pyDEW.Fluid(system, T=T, P=P, molalities=molalities, uacion='CL-')
    print("Successfully created a fluid...")

    # Now run the equilibration calculation. This proceeds by
    # reacting the bulk composition defined with the fluid. Sometimes
    # it crashes if you ask it to do too many reaction steps, which
    # I have tried to catch with the try, except blocks
    try:
        print("Attempting equilibrium calculation for zimax=300...")
        reaction = run_reaction(fluid, T, P, bulk_composition, 300)
    except Exception:
        print("zimax=300 did not work, trying zimax=200...\n")
        try:
            reaction = run_reaction(fluid, T, P, bulk_composition, 200)
        except Exception:
            print("zimax=200 did not work, trying zimax=100...\n")
            try:
                reaction = run_reaction(fluid, T, P, bulk_composition, 100)
            except Exception:
                assert False, "The calculation is not working under these "\
                              "conditions. Try adjusting the pressure and "\
                              "temperature a bit, or move to the next point."
    
    return reaction
            
def calculate_assemblage(T, P, bulk_composition):

    print(f"Starting calculation at {T:.0f}˚C and {P/1000:.1f} kbar...")

    # Convert degC to K
    T += 273.15

    # Import the table of mineral formulae
    mineral_formulae = pd.read_csv('mineral_parameters.csv')

    # Run the calculation
    reaction = run_model(T, P, bulk_composition)

    # Extract mineral information
    minerals = reaction.minerals.iloc[-1][~np.isnan(reaction.minerals.iloc[-1])].to_dict()
    solidsolutions = reaction.solid_solutions.iloc[-1].to_dict()

    totmoles = 0.0
    for mineral_name in minerals:
        totmoles += 10**minerals[mineral_name]
    
    print("")
    print(f"{f'Mineral' : <20} Mole Fraction   Molar Mass       Formula")
    print("="*85)
    for mineral_name in minerals:
        moles = 10**(minerals[mineral_name]) 
        
        if mineral_name[-4:] != '(SS)':
            formula = mineral_formulae[mineral_formulae.name==mineral_name].formula.iloc[0]
            molar_mass = np.sum(pyDEW.phase_models.get_mineral(mineral_name).element_comp[:27] * atm_wt)

            print(f"{f'{mineral_name}' : <24} {f'{moles/totmoles:.3f}' : <5}         {f'{molar_mass:.2f}' : <8}       {formula}")
        else:
            print(f"{f'{mineral_name}' : <24} {f'{moles/totmoles:.3f}' : <5}")
            print("-"*85)
            endmembers = defaultsystem.solid_solutions[mineral_name]
            for endmember_name in endmembers:
                emx = solidsolutions[endmember_name]
                formula = mineral_formulae[mineral_formulae.name==endmember_name].formula.iloc[0]
                molar_mass = np.sum(pyDEW.phase_models.get_mineral(endmember_name).element_comp[:27] * atm_wt)
                print(f"    {f'{endmember_name}' : <20} {f'{emx:.3f}' : <5}         {f'{molar_mass:.2f}' : <8}       {formula}")
        print("")
    
    print("")
    print("The Aqueous Species in the Fluid")
    print("")
    print("Species          Molality (mol/kg of H2O solvent)")
    print("="*45)
    species = reaction.species_concs.iloc[-2]
    print(10**species[species>-6])


## Calculations

First, set the bulk composition. This bulk composition is a typical mid-ocean ridge basalt.

In [56]:
# The units here are weight percent (wt%)
bulk_composition = {'SiO2': 47.4, 
                    'Al2O3': 17.64, 
                    'FeO': 7.98, 
                    'MgO': 7.63, 
                    'CaO': 12.44, 
                    'K2O': 0.03, 
                    'Na2O': 2.65, 
                    }

Now you can run a calculation at the temperature and pressure you are interested in. It should print out a results table.

In [57]:
calculate_assemblage(100.0, # Temperature in ˚C
                     5000.0, # Pressure in bar
                     bulk_composition)

Starting calculation at 100˚C and 5.0 kbar...
Successfully created a fluid...
Attempting equilibrium calculation for zimax=1000...

Mineral              Mole Fraction   Molar Mass       Formula
CHLORITE(SS)             0.320
-------------------------------------------------------------------------------------
    CLINOCHLORE          0.811         111.16         Mg(1)Al(0.4)Si(0.6)O(3.6)H(1.6)
    CHAMOSITE            0.189         142.70         Fe(1)Al(0.4)Si(0.6)O(3.6)H(1.6)

CPX_SUBCALCIC(SS)        0.379
-------------------------------------------------------------------------------------
    DIOPSIDE             0.235         216.55         Mg(1)Ca(1)Si(2)O(6)
    HEDENBERGITE         0.380         248.09         Ca(1)Fe(1)Si(2)O(6)
    ENSTATITE_CL         0.000         200.77         Mg(2)Si(2)O(6)
    JADEITE              0.384         202.14         Na(1)Al(1)Si(2)O(6)

HEMATITE                 0.000         159.69         Fe(2)O(3)

KFELDSPAR(SS)            0.013
-----------